In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets

import numpy as np
import pandas as pd
from tqdm import tqdm
import os

c:\Projects\stat-learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


# Define MTCNN baseline
We use the default params for now

In [3]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    keep_all=True, device=device
)

# Define Dataset

In [4]:
data_dir = "data/test"

try:
    # create testing folder
    os.makedirs(data_dir)

    # create label folders
    os.makedirs(f"{data_dir}/face")
    os.makedirs(f"{data_dir}/no_face")
except:
    print("Folders already exist.")

Folders already exist.


In [5]:
labels = pd.read_csv("labels.csv")

In [30]:
# Get all image names

files = []

for (dirpath, dirnames, filenames) in os.walk("data"):
    files.extend(filenames)
    break

files

['4bjXGaAEkgPqC7o1Q8wbnc2fxVX2.webp',
 '4bohll9CiZbKU2x2VYr3d1on8Nc2.webp',
 '4bu3aNg4BdXk6cD4vLrVS7WJPj63.webp',
 '4bwy3uZZJpdElOg2mSrlIz6MM8a2.webp',
 '4c05qi060lPLkbgDZN0KT4FyJKv2.webp',
 '4c2YX9OuP1c5rEtilpcQWuMn55B2.webp',
 '4c3WSwOjMvYMh3638l54avDZfxC3.webp',
 '4c4HCSjXBPd4kTIyJxs9U9CQHr32.webp',
 '4caadIduIWUtIW2MYxVOG5QaxlE2.webp',
 '4cabHD1OGIafsu56BPkdWDcmJuH3.webp',
 '4ccR11ee95gDPlBFGvZ7Ggndacd2.webp',
 '4cE0Xar5pVQ50ZuBYGzOECazUqu1.webp',
 '4cFMeng0xLTQUEUcZdzf0JkndMh2.webp',
 '4cgntZzyKZMRdDEbK3SLz03tJ1K2.webp',
 '4chdRi1ellWPm8AbifbzbtaJNkb2.webp',
 '4cInMh2ZU4hhZn1IkGblYWSlpI62.webp',
 '4cIZFenZE2UMTIkrm1USQra2aQf1.webp',
 '4cJacBMs8menRO19pmqnLc6PXZE3.webp',
 '4cjRqHYNdqP2DUlM0jLBAXUfMOJ3.webp',
 '4corbXjRTveWxArZkwacexThIzj1.webp',
 '4cqS6Vctxdh3Hpc8UKvz72WIRt03.webp',
 '4csjZliEm9a6nEua69J8Nr8RFYk2.webp',
 '4csr1Hel3Qe4xQe26YeaAxAJoI52.webp',
 '4cV3wsKlMDXcsHlAckAsOBF0U643.webp',
 '4czIxd3kIqgoCIqQlY7VT8ZNer03.webp',
 '4ddMp0rFXdSo8iJU0U63dHhoj4O2.webp',
 '4dFevG7vnt

In [ ]:
import ipywidgets as widgets
from IPython.display import Image, display, clear_output

face_bttn = widgets.Button(description="Face")
no_face_bttn = widgets.Button(description="No Face")
out = widgets.Output()

count = [0]

curr_file = ''

def face_bttn_clicked(_):
    d = {'filename': files[0],
                   'label': 'face'}
    files.pop(0)
    labels.insert(d)

    show_widgets()
        
face_bttn.on_click(face_bttn_clicked)

def no_face_clicked(_):
    d = {'filename': files[0],
                   'label': 'no face'}
    files.pop(0)
    labels.insert(d)

    show_widgets()

no_face_bttn.on_click(no_face_clicked)

def show_widgets():
    clear_output(wait=True)
    buttons = widgets.HBox([face_bttn, no_face_bttn])
    
    image = widgets.Image(
        value=Image(filename=f"data/{files[0]}").data,
        format="webp",
        width=300,
        height=300
    )
    
    text = widgets.Text(f"Total labeled: {len(labels)}")
    
    display(widgets.VBox([buttons, image, out]))
    
    
show_widgets()

In [ ]:
labels.to_csv("new_labels.csv", index=True)

In [6]:
def test(row):
    filename = row["filename"]
    label = row["label"].replace(" ", "_")
    
    os.rename(f"data/{filename}", f"{data_dir}/{label}/{filename}")

faces = labels[labels["label"] == 'face']
no_faces = labels[labels["label"] == 'no face']

try:
    faces.apply(test, axis=1)
    no_faces.apply(test, axis=1)
    
    print("Moved files to relevant folders")
except:
    print("Images are already moved")

Images are already moved


In [7]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('data/test')
idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn)

In [8]:
aligned = []
names = []

accurate = 0
for x, y in tqdm(loader):
    x_aligned = mtcnn.detect(x)
    
    if x_aligned[0] is not None and y == 0:
        accurate += 1
        
f"Accuracy is: {accurate / len(faces):8f}"


100%|██████████| 6039/6039 [14:04<00:00,  7.15it/s]


'Accuracy is: 0.928238'

NameError: name 'true_positives' is not defined